In [14]:
import json
import random

def generateMap(size, difficulty):
    map = [[0] * size for _ in range(size)]
    rng = random.Random()
    
    # Place mines randomly
    for _ in range(difficulty):
        while True:
            row, col = rng.randint(0, size-1), rng.randint(0, size-1)
            if map[row][col] != -1:
                map[row][col] = -1
                break
    
    # Calculate adjacent mines for each cell
    for i in range(size):
        for j in range(size):
            if map[i][j] != -1:
                map[i][j] = sum(1 for di in [-1, 0, 1] for dj in [-1, 0, 1] 
                                if 0 <= i+di < size and 0 <= j+dj < size and map[i+di][j+dj] == -1)
    
    return map

def update_game(actionString, gameState):
    action = json.loads(actionString)
    row, col = action["row"], action["col"]
    
    # Check if action is to flag
    if action.get("flag"):  
        gameState[row][col] = 9  # Represent flagged cells with 9
    else:
        if gameState[row][col] == -1:  # If it's a mine
            print("Game Over")
            gameState[row][col] = -3  # Indicate the mine was triggered
        elif gameState[row][col] >= 0:  # Reveal number of adjacent mines
            gameState[row][col] = count_adjacent_mines(row, col, gameState)

    return gameState

def count_adjacent_mines(row, col, gameState):
    """Count mines in adjacent cells."""
    size = len(gameState)
    return sum(1 for di in [-1, 0, 1] for dj in [-1, 0, 1] 
               if 0 <= row+di < size and 0 <= col+dj < size and gameState[row+di][col+dj] == -1)

# Function to print the game state neatly
def print_game_state(gameState, state_description):
    print(f"{state_description} Game State:")
    for row in gameState:
        print(" ".join(str(cell).rjust(2) for cell in row))
    print()

# Initialize game
size = 5
difficulty = 5
gameState = generateMap(size, difficulty)

# Print initial game state
print_game_state(gameState, "Initial")

# Example JSON action strings (You can send these to ChatGPT)
revealActionString = '{"row": 2, "col": 2, "flag": false}'
flagActionString = '{"row": 1, "col": 2, "flag": true}'

# Update game state with the actions
gameState = update_game(revealActionString, gameState)
print_game_state(gameState, "After Reveal")

gameState = update_game(flagActionString, gameState)
print_game_state(gameState, "After Flagging")


Initial Game State:
-1  1  2 -1  2
 1  2  3 -1  2
 0  1 -1  2  1
 0  1  2  2  1
 0  0  1 -1  1

Game Over
After Reveal Game State:
-1  1  2 -1  2
 1  2  3 -1  2
 0  1 -3  2  1
 0  1  2  2  1
 0  0  1 -1  1

After Flagging Game State:
-1  1  2 -1  2
 1  2  9 -1  2
 0  1 -3  2  1
 0  1  2  2  1
 0  0  1 -1  1

